##Домашняя работа

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),<br>
("2023-11-21", "Electronics", 110, 13000),<br>
("2023-11-22", "Electronics", 105, 12500),<br>
("2023-11-20", "Clothing", 300, 15000),<br>
("2023-11-21", "Clothing", 280, 14000),<br>
("2023-11-22", "Clothing", 320, 16000),<br>
("2023-11-20", "Books", 150, 9000),<br>
("2023-11-21", "Books", 200, 12000),<br>
("2023-11-22", "Books", 180, 10000)<br>

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [ ]:
!pip install pyspark >> None

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName("product_sales").getOrCreate()

# Загрузка исходных данных
data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])

df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [ ]:
# С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

# Оконная функция для нахождения среднего значения выручки от продаж
avg_sales_window = Window.partitionBy("category")

# Добавление столбца со средними значениями выручки от продаж для каждой категории продукта.
df = df.withColumn("avg_sales", F.avg("revenue").over(avg_sales_window))

df.show()

+----------+-----------+--------+-------+------------------+
|      date|   category|quantity|revenue|         avg_sales|
+----------+-----------+--------+-------+------------------+
|2023-11-20|      Books|     150|   9000|10333.333333333334|
|2023-11-21|      Books|     200|  12000|10333.333333333334|
|2023-11-22|      Books|     180|  10000|10333.333333333334|
|2023-11-20|   Clothing|     300|  15000|           15000.0|
|2023-11-21|   Clothing|     280|  14000|           15000.0|
|2023-11-22|   Clothing|     320|  16000|           15000.0|
|2023-11-20|Electronics|     100|  12000|           12500.0|
|2023-11-21|Electronics|     110|  13000|           12500.0|
|2023-11-22|Electronics|     105|  12500|           12500.0|
+----------+-----------+--------+-------+------------------+



In [ ]:
# Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом,
# чтобы в качестве строк были категории продуктов, в качестве столбцов были дни,
# а значениями были средние значения выручки от продаж за соответствующий день

# Применение функции pivot
pivot_df = df.groupBy("category").pivot("date").agg(F.avg("revenue"))

# Вывод средних значений выручки категории продуктов от продаж за соответствующий день
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

